In [91]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

train_data = pd.read_csv('rating.csv')
user_ids_array = train_data['userid'].unique()

In [92]:
def collaborative_filtering_recommendations(train_data, target_user_id, top_n=10):
    # Create the user-item matrix
    user_item_matrix = train_data.pivot_table(index='userid', columns='productid', values='rating', aggfunc='mean').fillna(0)
    
    # Calculate the user similarity matrix using cosine similarity
    user_similarity = cosine_similarity(user_item_matrix)

    # Find the index of the target user (assuming target_user_id is a valid index in user_item_matrix)
    try:
        target_user_index = user_item_matrix.index.get_loc(target_user_id)
    except KeyError:
        raise ValueError(f"User ID {target_user_id} not found in the dataset.")

    # Get the similarity scores for the target user
    user_similarities = user_similarity[target_user_index]

    # Sort the users by similarity in descending order (excluding the target user)
    similar_users_indices = user_similarities.argsort()[::-1][1:]

    # Generate recommendations based on similar users
    recommended_items = []

    for user_index in similar_users_indices:
        # Get items rated by the similar user but not by the target user
        rated_by_similar_user = user_item_matrix.iloc[user_index]
        not_rated_by_target_user = (rated_by_similar_user > 0) & (user_item_matrix.iloc[target_user_index] == 0)

        # Extract the item IDs of recommended items
        recommended_items.extend(user_item_matrix.columns[not_rated_by_target_user][:top_n])

    # Get the details of recommended items
    recommended_items_details = train_data[train_data['productid'].isin(recommended_items)][['productid', 'rating']]

    return recommended_items_details

In [93]:
target_user_id_index = 1
target_user_id = user_ids_array[target_user_id_index]
collaborative_filtering_rec = collaborative_filtering_recommendations(train_data, target_user_id)
print(f"Recommendations for User {user_ids_array[target_user_id_index]}:")
collaborative_filtering_rec

Recommendations for User 68271:


,productid,rating
0,hkt242817,2.0
2,msh089295,4.9
3,str226246,1.6
4,tvc114892,1.3
5,app176509,2.6
...,...,...
5995,mcl107836,2.5
5996,msh085527,3.3
5997,wcl036736,1.7
5998,app137954,1.3
